In [1]:
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup
import json
import random

In [3]:
#***          Start ib_insync (once)       *****
#_______________________________________________

from ib_insync import *
util.startLoop()
ib=IB().connect('127.0.0.1', 7496, clientId=3) # kavi TWS live
# ib=IB().connect('127.0.0.1', 4001, clientId=3) # kavi IBG live

In [4]:
#******   Error catch in list comprehension  ****
#________________________________________________

def catch(func, handle=lambda e : e, *args, **kwargs):
    '''List comprehension error catcher'''
    try:
        return func(*args, **kwargs)
    except Exception as e:
        return np.nan

In [5]:
#******          Symbols Master      *******
#___________________________________________


# NSE symbols from https://www.nseindia.com/products/content/derivatives/equities/fo_underlying_home.htm

index_symbols = 'FTSE100,INDIAVIX,NIFTYCPSE,NIFTY,NIFTYIT,BANKNIFTY,NIFTYMID50,NIFTYPSE,NIFTYINFRA'.split(',')

symbols='ACC,ADANIENT,ADANIPORTS,ADANIPOWER,AJANTPHARM,ALBK,AMARAJABAT,AMBUJACEM,ANDHRABANK,APOLLOHOSP,\
APOLLOTYRE,ARVIND,ASHOKLEY,ASIANPAINT,AUROPHARMA,AXISBANK,BAJAJ-AUTO,BAJFINANCE,BAJAJFINSV,\
BALKRISIND,BALRAMCHIN,BANKBARODA,BANKINDIA,BATAINDIA,BEML,BERGEPAINT,BEL,BHARATFIN,BHARATFORG,\
BPCL,BHARTIARTL,INFRATEL,BHEL,BIOCON,BOSCHLTD,BRITANNIA,CADILAHC,CANFINHOME,CANBK,CAPF,CASTROLIND,\
CEATLTD,CENTURYTEX,CESC,CGPOWER,CHENNPETRO,CHOLAFIN,CIPLA,COALINDIA,COLPAL,CONCOR,CUMMINSIND,DABUR,\
DALMIABHA,DCBBANK,DHFL,DISHTV,DIVISLAB,DLF,DRREDDY,EICHERMOT,ENGINERSIN,EQUITAS,ESCORTS,EXIDEIND,\
FEDERALBNK,GAIL,GLENMARK,GMRINFRA,GODFRYPHLP,GODREJCP,GODREJIND,GRANULES,GRASIM,GSFC,HAVELLS,HCLTECH,\
HDFCBANK,HDFC,HEROMOTOCO,HEXAWARE,HINDALCO,HCC,HINDPETRO,HINDUNILVR,HINDZINC,ICICIBANK,ICICIPRULI,IDBI,\
IDEA,IDFCBANK,IDFC,IFCI,IBULHSGFIN,INDIANB,IOC,IGL,INDUSINDBK,INFIBEAM,INFY,INDIGO,IRB,ITC,JISLJALEQS,\
JPASSOCIAT,JETAIRWAYS,JINDALSTEL,JSWSTEEL,JUBLFOOD,JUSTDIAL,KAJARIACER,KTKBANK,KSCL,KOTAKBANK,KPIT,\
L%26TFH,LT,LICHSGFIN,LUPIN,M%26MFIN,MGL,M%26M,MANAPPURAM,MRPL,MARICO,MARUTI,MFSL,MINDTREE,MOTHERSUMI,\
MRF,MCX,MUTHOOTFIN,NATIONALUM,NBCC,NCC,NESTLEIND,NHPC,NIITTECH,NMDC,NTPC,ONGC,OIL,OFSS,ORIENTBANK,PAGEIND,\
PCJEWELLER,PETRONET,PIDILITIND,PEL,PFC,POWERGRID,PTC,PNB,PVR,RAYMOND,RBLBANK,RELCAPITAL,RCOM,RELIANCE,\
RELINFRA,RPOWER,REPCOHOME,RECLTD,SHREECEM,SRTRANSFIN,SIEMENS,SREINFRA,SRF,SBIN,SAIL,STAR,SUNPHARMA,SUNTV,\
SUZLON,SYNDIBANK,TATACHEM,TATACOMM,TCS,TATAELXSI,TATAGLOBAL,TATAMTRDVR,TATAMOTORS,TATAPOWER,TATASTEEL,TECHM,\
INDIACEM,RAMCOCEM,SOUTHBANK,TITAN,TORNTPHARM,TORNTPOWER,TV18BRDCST,TVSMOTOR,UJJIVAN,ULTRACEMCO,UNIONBANK,\
UBL,MCDOWELL-N,UPL,VEDL,VGUARD,VOLTAS,WIPRO,WOCKPHARMA,YESBANK,ZEEL'.split(',')

In [5]:
#......    !Data Limiter   .........
symbols = random.sample(symbols,5)
#___________________________________

In [7]:
# Rectified for IBKR
isymbols = [i[0:9] for i in symbols]
isymbols = [s.replace('%26','') for s in isymbols]

# Rectified for 5paisa
psymbols = [s.replace('%26', '&') for s in symbols]

In [13]:
%%time
#***     Dictionaries and extracts ***
#_____________________________________

# URLs
urlbase = "https://www.nseindia.com/live_market/dynaContent/live_watch/"
nseurl = urlbase + "get_quote/GetQuote.jsp?symbol="

# Scrape
nsestocks = [nseurl+i for i in symbols]
nserequests = [requests.get(b).text for b in nsestocks]
bs_nses = [BeautifulSoup(s, 'html.parser') for s in nserequests]
nse_data = [f.find(id='responseDiv').text.strip() for f in bs_nses]
nse_dict = [json.loads(bd) for bd in nse_data]

# NSE dictionary extracts
nse_margin = {i['data'][0]['symbol']: float(i['data'][0]['applicableMargin'])/1000 for i in nse_dict}
nse_hi52 = {i['data'][0]['symbol']: float((i['data'][0]['high52']).replace(',','')) for i in nse_dict}
nse_lo52 = {i['data'][0]['symbol']: float((i['data'][0]['low52']).replace(',','')) for i in nse_dict}
nse_lastprice = {i['data'][0]['symbol']: float((i['data'][0]['lastPrice']).replace(',','')) for i in nse_dict}

# MLot and TotalMargin from 5paisa
paisaurl = "https://www.5paisa.com/5pit/spma.asp"
paisa = pd.read_html(paisaurl, header=0)[1].drop_duplicates(subset='Symbol').reset_index(drop=True)
paisa_small = paisa[['Symbol', 'Mlot', 'TotMgnPerLt']].sort_values('Symbol')

paisa_dict = paisa_small.set_index('Symbol').T.to_dict()

Wall time: 5.5 s


In [14]:
%%time
#***   Determine Standard Deviation    ***
#_________________________________________

market = 'NSE'

# qualify contracts
contracts = [Stock(symbol=s, exchange=market) for s in isymbols]
qual_stocks = ib.qualifyContracts(*contracts)

# make a historical dictionary
hist_dict = dict([(c.symbol, ib.reqHistoricalData(contract=c, endDateTime='', durationStr='365 D', 
                                    barSizeSetting='1 day',  whatToShow='Trades', useRTH=True)) for c in contracts])

data = [v for k, v in hist_dict.items()]

sd252 = [catch(lambda: np.std([i.close for i in data[j]][-252:], ddof=1)) 
         for j, k in enumerate(data)] # 252 trading days in a year

sd252_dict = dict(zip(isymbols, sd252))

Wall time: 3.96 s


In [ ]:
%%time
#***    Get the RoM  ****
#________________________

# Option chains for qualified stocks
id_symbols = [(s.conId, s.symbol) for s in qual_stocks]

chains = [(symbol, 
           ib.reqSecDefOptParams(futFopExchange='', underlyingConId=cid, underlyingSecType='STK', underlyingSymbol=symbol)) 
          for cid, symbol in id_symbols]

str_exp_dict = dict([(x[0], (sorted(x[1][0].expirations), sorted(x[1][0].strikes))) for x in chains])


# Conversion dictionary for symbols
i2n_symbols = dict(zip(isymbols, psymbols)) #psymbols used because '%26' is converted to '&'...for nse_lastprice

# Filter the strikes
puts_list = [{k: (v[0], i)} 
             for k, v in str_exp_dict.items() 
             for i in v[1] 
             if i < (nse_lastprice[i2n_symbols[k]]- sd252_dict[k]*2)]

calls_list = [{k: (v[0], i)} 
             for k, v in str_exp_dict.items() 
             for i in v[1] 
             if i > (nse_lastprice[i2n_symbols[k]] + sd252_dict[k]*2)]

puts_clist = [{k: ('NSE', e, v[1], 'P')} for puts_dict in puts_list for k, v in puts_dict.items() for e in v[0]]
calls_clist = [{k: ('NSE', e, v[1], 'C')} for calls_dict in calls_list for k, v in calls_dict.items() for e in v[0]]

put_option_contracts = [Option(symbol=k, exchange=v[0], lastTradeDateOrContractMonth=v[1], strike=v[2], right=v[3]) 
for puts_dict in puts_clist for k, v in puts_dict.items()]


# Qualify the put contracts
qual_puts = [ib.qualifyContracts(c) for c in put_option_contracts]

qual_puts = [l for i in qual_puts for l in i] # Remove the empty [] entries

live_put_ticks = [t for i in range(0, len(qual_puts), 100) for t in ib.reqTickers(*qual_puts[i:i+100])]

In [30]:
for l in live_put_ticks:
    print(l.close, l.bid, l.ask)

0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.35 -1.0 -1.0
1.55 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0
0.0 -1.0 -1.0


In [35]:
ib.reqMktData(qual_puts[47], "", False, False)

Ticker(contract=Option(conId=335420828, symbol='EQUITAS', lastTradeDateOrContractMonth='20181227', strike=102.5, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='EQUITAS18DEC102.5PE', tradingClass='EQUITAS'), time=datetime.datetime(2018, 9, 30, 13, 22, 12, 565803, tzinfo=datetime.timezone.utc), bid=-1.0, bidSize=0, ask=-1.0, askSize=0, close=0.0, ticks=[], tickByTicks=[], domBids=[], domAsks=[], domTicks=[])

# Things to do...
* get option prices
* make qual_calls
* merge with margin to find RoM
* sort on RoM
* place orders
* pickle!

In [ ]:
ib.disconnect()